I am trying to change the codes (4) in my downloads, to improve 
    1- the hard-coded directory names to a more relative path with the use of os.getcwd() (from the website 'tutorialspoint.com'). Otherwise, the codes cannot be functional on any other device, and not mine too!
    2- the problem with dumping files which seems to be based on an indentation imprecision. For this I just played around with indentations and lines of codes. 

but then, apparently I do not have the right input file. So, let's discuss this in the course! :)


In [3]:
import os
import glob
import pandas as pd
import pickle

# Get the current directory
current_directory = os.getcwd()

# Define the directory containing the input files
input_directory = os.path.join(current_directory, "parsed_data")

# Go through the files in the input directory
for filename in glob.glob(os.path.join(input_directory, '*.txt')):
    # List to store ALL nps of ONE FILE in
    nps_of_file = list()

    sentence_counter = 1
    np_counter = 1

     # Open the file in read-mode
    with open(filename, 'r', encoding="UTF-8-sig") as file:
        # Read in the lines of token
        file_input = file.readlines()

        # List to generate a single sentence out of the file
        single_sentence = list()

        # Go through each line/"sentence" of the file
        for line in file_input:
            # If it is not an empty line/"sentence", add it to the list of a single sentence
            if line[0] != "\n":
                single_sentence.append(line)

            # Empty line/"sentence" indicates the end of a sentence
            elif line[0] == "\n":
                # Split each element of the sentence into the single token-information
                sentence_as_elements = list()
                for row in single_sentence:
                    elements = row.split("\t")
                    sentence_as_elements.append(elements)

                # Create DataFrame with column names
                df = pd.DataFrame(sentence_as_elements, columns=["ID", "token", "lemma", "wordtype", "POSTag", "features", "head", "deprel", "dunno1", "dunno2"])

                # Delete columns we do not need
                df = df.drop(["dunno1", "dunno2"], axis=1)

                # Extract the whole token-column to get the whole sentence as tokens
                sentence_list = df["token"].values.tolist()
                sentence_as_string = " ".join(sentence_list)
                number_of_tokens_of_sentence = len(sentence_list)

                token_head_list = list()
                token_index_list = list()
                token_id_list = list()

                # Generate dictionary
                # Go through the DataFrame of the file
                for index, row in df.iterrows():
                    token_head_list.append([[row["token"]], row["head"]])

                    # Find the root
                    if row["deprel"] == "root":
                        if str(row["deprel"]).isalpha():
                            root_of_sentence = df.loc[index, :].values.flatten().tolist()
                        else:
                            if root_of_sentence:
                                root_of_sentence = "None"
                            else:
                                pass

                    # Add the token and its index to the other dictionary
                    token_index_list.append([[row["token"]], index])
                    token_id_list.append([[row["token"]], row["ID"]])

                # List for all NPs in the sentence
                all_nps_of_sentence = list()

                # Go through the DataFrame once again
                for index, row in df.iterrows():
                    # If the wordtype is a noun
                    if row["wordtype"] == "N":
                        # Create list for the NP:
                        current_np_ids = list()
                        # Get all the info of the noun
                        info_of_noun = df.loc[index, :].values.flatten().tolist()
                        id_of_noun = info_of_noun[0]
                        head_of_noun_ID = info_of_noun[6]
                        # Append the noun's ID to the list
                        current_np_ids.append(id_of_noun)
                        if int(head_of_noun_ID) != 0:
                            current_np_ids.append(head_of_noun_ID)

                        # In the following:
                        # Check if any word refers to the noun as head
                        for ind, token_head_pair in enumerate(token_head_list):
                            # Check if the ID of the noun is the deprel of another token --> noun is head of that token
                            if id_of_noun == token_head_pair[1]:
                                # Append the ID of the token of the sentence
                                # --> Get a list with IDs, can sort them and then create a second list with the filled-in token (information)
                                current_np_ids.append(token_id_list[ind][1])

                        current_np_ids.sort()
                        number_of_tokens_of_np = len(current_np_ids)

                        # Create NP with the regarding token information, based on the IDs
                        current_np = list()
                        for token_id in current_np_ids:
                            index_in_df = int(token_id) - 1
                            if int(index_in_df) < 0:
                                token_info = tuple(df.loc[int(token_id), :].values.flatten().tolist())
                            else:
                                token_info = tuple(df.loc[int(index_in_df), :].values.flatten().tolist())
                            current_np.append(token_info)

                        """
                        Create the final form of the single np on form of:
                        [   Satz_NP-Nummer,
                            Dateiname {str},
                            Satz_ID_NR {int},
                            Satz {str},
                            (root, POS-Tag, grammatical_info) {tuple},
                            Anzahl-der-Token-der-NP {int},
                            [NP] {list}, Bsp.: [(token1, POS-tag, grammatical_info),... (token-n, POS-tag, grammatical_info)],
                        """
                        sentence_np_id = str(sentence_counter) + "_" + str(np_counter)
                        final_form_of_np = [sentence_np_id, filename, sentence_counter, sentence_as_string, root_of_sentence, number_of_tokens_of_np, current_np]
                        np_counter += 1
                        all_nps_of_sentence.append(final_form_of_np)

                # Clear the list of the single sentence for the new sentence
                single_sentence = list()
                sentence_counter += 1
                nps_of_file.extend(all_nps_of_sentence)

    filename_2_0 = os.path.splitext(filename)[0]
    newfilename = filename_2_0 + "_exported_nps.txt"

    # Print each individual element of nps_of_file to the new txt-file
    new_filepath = os.path.join(newfilename)
    with open(new_filepath, "a", encoding="UTF-8-sig") as f:
        for nps in nps_of_file:
            for item in nps:
                print(item, file=f)
            print(file=f)

    # Save as a pickle file
    pickle_filename = filename_2_0 + "_exported_nps.pkl"
    pickle_filepath = os.path.join(pickle_filename)
    with open(pickle_filepath, 'wb') as f:
        pickle.dump(nps_of_file, f)
